In [1]:
import pandas as pd
import numpy as np

In [52]:
from nltk import tokenize
from operator import itemgetter
import math
from tqdm import tqdm
import os
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
stop_words = set(stopwords.words('english'))

In [3]:
kk = os.listdir("/home/amansinha/extractpubmed/papers/key_meta_all/")
kk

['tdm1_metadf2.csv',
 'genome wide association study_metadf2.csv',
 'progression free survival_metadf2.csv',
 'hormone therapy_metadf2.csv',
 'overall survival_metadf2.csv',
 'disease free survival_metadf2.csv',
 'radiation_metadf2.csv',
 'chemotherapy_metadf2.csv',
 'neratinib_metadf2.csv',
 'pertuzumab_metadf2.csv',
 'pathologic complete response_metadf2.csv',
 'surgery_metadf2.csv',
 'polymorphism_metadf2.csv',
 'neoadjuvant_metadf2.csv',
 'adjuvant_metadf2.csv',
 'trastuzumab_metadf2.csv',
 'polygenic risk score_metadf2.csv',
 'Breast Cancer_metadf2.csv']

In [4]:
def get_keywords(doc, k):
    total_words = doc.split()
    total_word_length = len(total_words)
    #print(total_word_length)
    
    total_sentences = tokenize.sent_tokenize(doc)
    total_sent_len = len(total_sentences)
    #print(total_sent_len)
    
    tf_score = {}
    for each_word in total_words:
        each_word = each_word.replace('.', '')
        if each_word in tf_score:
            tf_score[each_word] += 1
        else:
            tf_score[each_word] = 1
            
    tf_score.update((x, y/int(total_word_length)) for x,y in tf_score.items())
    #print(tf_score)
    
    def check_sent(word, sentences): 
        final = [all([w in x for w in word]) for x in sentences] 
        sent_len = [sentences[i] for i in range(0, len(final)) if final[i]]
        return int(len(sent_len))
    
    idf_score = {}
    for each_word in total_words:
        each_word = each_word.replace('.','')
        if each_word not in stop_words:
            if each_word in idf_score:
                idf_score[each_word] = check_sent(each_word, total_sentences)
            else:
                idf_score[each_word] = 1

    # Performing a log and divide
    idf_score.update((x, math.log(int(total_sent_len)/y)) for x, y in idf_score.items())
    #print(idf_score)
    
    tf_idf_score = {key: tf_score[key] * idf_score.get(key, 0) for key in tf_score.keys()}
    #print(tf_idf_score)
    
    result = dict(sorted(tf_idf_score.items(), key = itemgetter(1), reverse = True)[:k]) 
    return result
    
    

In [5]:
df = pd.read_csv('/home/amansinha/extractpubmed/papers/key_meta_all/tdm1_metadf2.csv')
df

,pubmed_id,title,keywords,publication_date,abstract
0,33479246,Fibroblast growth factor receptor facilitates ...,[],2021-01-23,Trastuzumab-emtansine (T-DM1) is an antibody-d...
1,33259892,Third-line treatment of HER2-positive advanced...,"['Breast cancer', 'HER2\xa0+', 'Third line', '...",2020-12-02,Human epidermal growth factor receptor 2 posit...
2,33238772,Margetuximab for the treatment of HER2-positiv...,"['Margetuximab', 'breast cancer', 'her2', 'mga...",2020-11-27,No specific standard treatment is currently re...
3,32737515,Post-neoadjuvant treatment with capecitabine a...,"['Breast cancer', 'Capecitabine', 'Radiochemot...",2020-08-02,Following neoadjuvant chemotherapy for breast ...
4,32236828,Comparison of outcomes in a population-based c...,"['Cardiotoxicity', 'Heart failure', 'Overall s...",2020-04-03,Little data exist for comparing cardiac safety...
5,32192389,Safety of trastuzumab after trastuzumab emtans...,"['Breast cancer', 'HER2', 'nodular regenerativ...",2020-03-21,Trastuzumab emtansine is an antibody-drug conj...
6,32104023,Analysis of the Cost-Effectiveness of Liquid B...,"['biomarkers', 'breast cancer', 'cost-effectiv...",2020-02-28,Breast cancer is highly prevalent worldwide an...
7,31949426,Recent advances and optimal management of huma...,"['Breast cancer', 'TDM1', 'neratinib', 'pertuz...",2020-01-18,With the introduction of anthracycline-based r...
8,31781874,When to Add Additional Anti-HER2 Therapy to Ad...,"['Adjuvant', 'Early', 'HER2', 'Neoadjuvant', '...",2019-11-30,One year of trastuzumab dramatically improves ...
9,31449983,The evolving role of trastuzumab emtansine (T-...,"['Brain metastases', 'Breast cancer', 'HER2', ...",2019-08-27,Approximately 30-50% of advanced human epiderm...


In [53]:
tn, tt1, tt2, tt3, tabst1, tabst2,tabst3, tc = [],[],[],[], [], [], [], []
c = 0
for k in kk:
    
    df = pd.read_csv(f'/home/amansinha/extractpubmed/papers/key_meta_all/{k}')
    df['keywords'] = df['keywords'].str.strip('[]').str.split(',')
    df['keywords'] = df['keywords'].apply(lambda x: [xx.strip().strip("''") for xx in x])
    
    
    name = k.split('_')[0]

    ta1, ta2, ta3 = [], [], []
    
    for t in ['title', 'abstract']:
        c= 0
        scores1, scores2, scores3 = [], [],[]
        for i, (key, abst) in tqdm(enumerate(zip(df['keywords'], df[t]))):
            try:
                #print((key))
                if len(key)>1  and (not pd.isna(abst)):
                    keylist = set(list(map(str.lower, key)))
                    manual_keylist = set(list(map(str.lower, get_keywords(abst, 10).keys() )))
                    s1 = simple_match(keylist, manual_keylist)
                    s2 = substring_match(keylist, manual_keylist)
                    s3 = semantic_match(keylist, manual_keylist, tokenizer, model)
                    
                    #print(i, 'authorkeys:',keylist,'\nmanualkeys:',manual_keylist, '\n')
                    scores1.append(s1)
                    scores2.append(s2)
                    scores3.append(s3)
                    c +=1
            except:
                print(i, 'authorkeys:',keylist,'\nmanualkeys:',manual_keylist, '\n')
                print('ISSUE:', abst)
                break
            #if c == 100: break
                
        ta1.append(sum(scores1)/c)
        ta2.append(sum(scores2)/c)
        ta3.append(sum(scores3)/c)
        #print('*'*50)
        
        
    
    print(f'For {name}: title are {ta3[0]} and abstract are {ta3[1]}')
    tn.append(name); tt1.append(ta1[0]);tt2.append(ta2[0]); tt3.append(ta3[0]); \
    tabst1.append(ta1[1]); tabst2.append(ta2[1]);tabst3.append(ta3[1]); tc.append(c)
    #break
    #result.append({'key':name,'keywordVtitle_b': ta1[0], 'keywordVtitle_sbs': ta2[0], 'keywordVabstract_b': ta1[1],'keywordVabstract_sbs': ta2[1]}, ignore_index=True)
    
    

43it [00:12,  3.39it/s]
43it [00:12,  3.42it/s]
0it [00:00, ?it/s]

For tdm1: title are 0.5385959153340232 and abstract are 0.5577921007718477


5000it [27:17,  3.05it/s]
4629it [24:48,  3.11it/s]
0it [00:00, ?it/s]

4629 authorkeys: {'cholesky decomposition', 'multiphenotype analysis', 'genome-wide association study', 'kinship matrix', 'linear mixed models'} 
manualkeys: set() 

ISSUE: 
                    
For genome wide association study: title are nan and abstract are 0.557202183806945


4998it [1:04:46,  1.29it/s]
4998it [35:33,  2.34it/s]
0it [00:00, ?it/s]

...

8it [00:04,  1.99it/s]
0it [00:00, ?it/s]

8 authorkeys: {'trastuzumab deruxtecan', 'tucatinib', 'trastuzumab-emtansine', 'lapatinib', 'neratinib', 'trastuzumab', 'brain metastases', 'her2-positive breast cancer'} 
manualkeys: {'current', 'cancer', 'options', 'breast', 'her2-positive', 'with', 'for', 'brain', 'patients', 'treatment'} 

ISSUE: Current treatment options for HER2-positive breast cancer patients with brain metastases.


0it [00:00, ?it/s]

0 authorkeys: {'human epidermal growth factor receptor 2', 'breast neoplasms', 'neratinib', 'hormone receptor'} 
manualkeys: {'human', 'receptor', 'factor', 'growth', 'epidermal', '(hr),', 'pi3k/akt/mtor', 'receptor-2(her-2)', 'highly', 'hormone'} 

ISSUE: Hormone receptor (HR), human epidermal growth factor receptor-2(HER-2) double positive breast cancer is a highly aggressive disease, and it is more susceptible to drug resistance and metastasis. Currently, although anti-HER-2 monoclonal antibody combined with endocrine therapy is commonly used in clinical practice, patients with HR-positive and HER-2-positive still have a poor prognosis. That may be caused by the activation of downstream PI3K/Akt/mTOR signaling pathway of HER-2. Therefore, simultaneously blocking the extracellular, intracellular and downstream PI3K/Akt/mTOR signaling pathway of HER-2, as well as the dual blockade of HER-2 and HR, is more conducive to the treatment of HR positive and HER-2 positive breast cancer. The 

ZeroDivisionError: division by zero

In [54]:
result = pd.DataFrame(columns = ['key', 'keywordVtitle_b','keywordVtitle_sbs', 'keywordVtitle_sem', 'keywordVabstract_b','keywordVabstract_sbs', 'keywordVabstract_sem', 'nsamples'])
result['key']  = tn
result['keywordVtitle_b'] = tt1
result['keywordVtitle_sbs'] = tt2
result['keywordVtitle_sem'] = tt3
result['keywordVabstract_b'] = tabst1
result['keywordVabstract_sbs'] = tabst2
result['keywordVabstract_sem'] = tabst3
result['nsamples'] = tc

In [55]:
result

,key,keywordVtitle_b,keywordVtitle_sbs,keywordVtitle_sem,keywordVabstract_b,keywordVabstract_sbs,keywordVabstract_sem,nsamples
0,tdm1,0.112037,0.669167,0.538596,0.055952,0.377976,0.557792,28
1,genome wide association study,0.071084,0.761368,NaN,0.045866,0.356892,0.557202,2944
2,progression free survival,0.120741,0.882582,NaN,0.067391,0.445179,NaN,3607
3,hormone therapy,0.103923,0.853283,NaN,0.059205,0.378839,NaN,3280
4,overall survival,0.107009,0.880749,NaN,0.062405,0.399481,NaN,3557
5,disease free survival,0.101470,0.926997,0.539590,0.055107,0.442631,0.545831,3551
6,radiation,0.088845,0.881257,NaN,0.000000,0.057143,0.579042,5
7,chemotherapy,0.000000,0.833333,0.547600,0.166667,0.166667,0.580154,1


Correlation is defined the average of count of intersection of number of keywords between the actual keywords provided by the authors over complete number of samples.

In [56]:
authorkeys= {'t‑dm1', 'radiochemotherapy', 'radiotherapy', 'capecitabine', 'breast cancer'} 
manualkeys= {'with', 'emtansine', 'and', 'trastuzumab', 'post-neoadjuvant', 'treatment', 'in', 'breast', 'cancer', 'capecitabine'}


In [57]:
def simple_match(authorkeys, manualkeys):
    return (len(authorkeys & manualkeys)/len(authorkeys))

def substring_match(authorkeys, manualkeys):
    c = 0
    for ak in authorkeys:
        for mk in manualkeys:
            c += int(ak.find(mk)!= -1)
    
    return c/len(authorkeys)
    
def semantic_match(authorkeys, manualkeys, tokenizer, model):
    embdict = dict()
    
    for k in authorkeys | manualkeys:
        inputs = tokenizer.encode_plus(k)
        ids = torch.tensor(inputs['input_ids']).unsqueeze(0)
        mask = torch.tensor(inputs['attention_mask']).unsqueeze(0)
        with torch.no_grad():
            embed = model(ids)[0]
        #print(embed.shape)
        final = torch.mean(embed[:, 1:-1], dim=1).squeeze().detach().numpy()
        
        embdict[k] = final, np.linalg.norm(final)
    
    c= 0
    for ak in authorkeys:
        a = 0
        for mk in manualkeys:
            iak, imk = embdict[ak], embdict[mk]
            a += iak[0].dot(imk[0])/(iak[1]*imk[1])
        c += a/len(manualkeys)
    
    return c/len(authorkeys)
    


In [12]:
import torch
import numpy as np
from transformers import *

tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_cased')
model = AutoModel.from_pretrained('allenai/scibert_scivocab_cased')

Some weights of the model checkpoint at allenai/scibert_scivocab_cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [18]:
authorkeys =  {'trastuzumab deruxtecan', 'tucatinib', 'ds8201a', 'her2\\xa0+', 'breast cancer', 'third line'} 
manualkeys = {'from', 'of', 'breast', 'cancer:', 'standard', 'her2-positive', 'third-line', 'no', 'advanced', 'treatment'}

In [39]:
semantic_match(authorkeys, manualkeys, tokenizer, model)

0.4891712586085002